### Extracción de Balance Comercial
[Link](https://www.dane.gov.co/index.php/estadisticas-por-tema/comercio-internacional/balanza-comercial)

In [ ]:
from selenium.webdriver.common.by import By
from datetime import datetime
import selenium as sel
import pandas as pd
import zipfile
import time
import glob
import sys
import os

cwd = os.getcwd()
cwd = cwd.replace("Notebooks", "Scripts")
sys.path.insert(0, cwd.replace("\\\\", "\\"))
import base as bs

In [ ]:
def extraer_balance_comercial(start_date: str, end_date: str):
    """Función que ingresa a la página y descarga el balance comercial. 
    Extrae la información de interes y la almacena en el Data Frame.

    Args:
        start_date (str): Fecha desde la que se obtendrán los datos.
        end_date (str): Fecha hasta la que se obtendrán los datos.

    Returns:
        pd.DataFrame: Data Frame con 5 columnas:
        Date | Importaciones | Exportaciones | Balance | Granularidad | Unidad |
    """
    driver = bs.ejecutar_driver('https://www.dane.gov.co/index.php/estadisticas-por-tema/comercio-internacional/balanza-comercial')
    balance_comercial = driver.find_elements(By.XPATH, './/div[@class="item-page clearfix"]')[0]
    sec = balance_comercial.find_element(By.XPATH, './/div[@class="btn-group btn-group-justified"]')
    url_archivo = sec.find_element(By.XPATH, './/a').get_attribute('href')
    driver.get(url_archivo)
    time.sleep(10)
    
    lista_url = url_archivo.split('/')
    nombre_archivo = [file for file in lista_url if file.endswith('.xlsx')][0] 

    path_to_file = '/Users/'+os.getlogin()+'/Downloads/' + \
        nombre_archivo 
    
    df = pd.read_excel(path_to_file, header=8)
    df = df.dropna(inplace=False)
    df = df.rename(columns={'Unnamed: 0': 'Año'})

    df.loc[df.index[-1], 'Año'] = datetime.now().year
    df['Año'] =pd.to_numeric(df['Año'])


    aux_1 = start_date.split('-')
    aux_2 = end_date.split('-')


    if float(aux_1[2]) in df['Año']:
        start=aux_1[2]
    else:
        start=df['Año'].min()

    if float(aux_2[2]) in df['Año']:
        end=aux_2[2]
    else:
        end=df['Año'].max()


    df['Granularidad'] = 'Anual'
    df['Unidad'] = 'Millones de dólares FOB'
    df['Date'] = '31-12-'+df['Año'].map(str)

    df_final = df[(df['Año'] >= start) & (df['Año'] <= end)]
    df_final = df_final.drop('Año', axis=1)
                            
    # os.remove(path_to_file)
    
    return df_final

In [ ]:
print(extraer_balance_comercial('01-01-1990', '01-01-2040'))